# Setup (RUN EVERYTIME)

In [ ]:
# Use if Drive mounting bugs out (Also restart session & refresh page)
from google.colab import drive
drive.flush_and_unmount()

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Change to desired working directory
%cd /content/drive/MyDrive/DATASCI266

/content/drive/MyDrive/DATASCI266


# Step 1: Load In Dataset

In [ ]:
import json

with open("/content/drive/MyDrive/DATASCI266/SQuAD/train-v2.0.json", "r") as f:
    squad_dataset = json.load(f)

## Step 1.1: Convert SQuAD to RACE MC-style + Generate Splits

In [ ]:
import random
def convert_squad_to_race_style(squad_dataset):
    processed_examples = []

    for element in squad_dataset["data"]:
        for element_paragraph in element["paragraphs"]:
            context = element_paragraph["context"]
            qas = element_paragraph["qas"]

            # Hold all possible options, Has correct & distractor answers
            new_options = []
            for qa in qas:
                # Skip questions that can't be answered
                if qa["is_impossible"] == True:
                    continue
                for ans in qa["answers"]:
                    answer_text = ans["text"].strip()
                    if answer_text not in new_options:
                        new_options.append(answer_text)

            # Now convert each qa in this paragraph
            for qa in qas:
                # Skip questions that can't be answered
                if qa["is_impossible"] == True:
                    continue

                question = qa["question"]
                correct_answer = qa["answers"][0]["text"].strip()

                # Get wrong distractor answers by filtering out the correct answer
                wrong_answers = []
                for ans in new_options:
                    if ans != correct_answer:
                        wrong_answers.append(ans)

                        # Only want to get 3 wrong distractor answers
                        if len(wrong_answers) == 3:
                            break

                # Need to ignore current example if there are less than 3 wrong
                # answers because it will break
                if len(wrong_answers) < 3:
                    continue

                options = [correct_answer] + wrong_answers
                random.shuffle(options)
                label = options.index(correct_answer)

                race_like_example = {
                    "context": context,
                    "query": question,
                    "option_0": options[0],
                    "option_1": options[1],
                    "option_2": options[2],
                    "option_3": options[3],
                    "label": label
                }

                processed_examples.append(race_like_example)

    return processed_examples


squad_dataset = convert_squad_to_race_style(squad_dataset)

In [ ]:
squad_dataset[0] # Inspecting structure of single record

{'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
 'query': 'When did Beyonce start becoming popular?',
 'option_0': 'singing and dancing',
 'option_1': 'in the late 1990s',
 'option_2': 'Houston, Texas',
 'option_3': '2003',
 'label': 1}

In [ ]:
import random

# Seed for reproducibility + Shuffle
random.seed(26)
random.shuffle(squad_dataset)

num_examples = len(squad_dataset)
train_end = int(0.8 * num_examples)
val_end = int(0.9 * num_examples)

train_data = squad_dataset[0 : train_end]
val_data   = squad_dataset[train_end : val_end]
test_data  = squad_dataset[val_end :]

print("Total:", num_examples)
print("Train:", len(train_data))
print("Val:", len(val_data))
print("Test:", len(test_data))

Total: 75902
Train: 60721
Val: 7590
Test: 7591


In [ ]:
# Need proper Dataset object for split names ("train", "test", "validation")
from datasets import Dataset, DatasetDict

train_ds = Dataset.from_list(train_data)
val_ds   = Dataset.from_list(val_data)
test_ds  = Dataset.from_list(test_data)

squad_dataset = DatasetDict({
    "train": train_ds,
    "validation": val_ds,
    "test": test_ds,
})

# Step 2: Convert into Text Prompt For Qwen2.5

In [ ]:
letter_options = ["A", "B", "C", "D"]

def format_example_as_text_prompt_for_qwen(example):
    '''
    Want to return {"final_prompt": final_prompt} where final prompt
    looks like the below:

    Context:
    <context>


    Question:
    <question>


    Options:
    A. <option_0>
    B. <option_1>
    C. <option_2>
    D. <option_3>

    Answer with the letter (A, B, C, or D) only.
    Answer: <correct_letter>
    '''
    # Create list of available answer options
    options = []
    for i in range(4):
      options.append(example[f"option_{i}"])

    # Create string of letter_options with their options
    options_block = ""
    for i, opt in enumerate(options):
        options_block += f"{letter_options[i]}. {opt}\n"

    # Use "label" field (0,1,2,3) of example to get the correct letter option
    correct_letter = letter_options[example["label"]]

    # Construct prompt
    prompt = (
        f"Context:\n{example['context']}\n\n"
        f"Question:\n{example['query']}\n\n"
        f"Options:\n{options_block}\n\n"
        "Answer with the letter (A, B, C, or D) only."
    )
    final_prompt = prompt + "\nAnswer: " + correct_letter

    return {"final_prompt": final_prompt}

In [ ]:
squad_dataset = squad_dataset.map(format_example_as_text_prompt_for_qwen, batched=False)

Map:   0%|          | 0/60721 [00:00<?, ? examples/s]

Map:   0%|          | 0/7590 [00:00<?, ? examples/s]

Map:   0%|          | 0/7591 [00:00<?, ? examples/s]

# Step 3: Tokenize for Qwen2.5
- Qwen2.5-0.5B-Instruct
  - link: https://huggingface.co/Qwen/Qwen2.5-0.5B-Instruct
- https://huggingface.co/docs/transformers/tasks/language_modeling
- https://huggingface.co/learn/llm-course/en/chapter7/6?utm_source=chatgpt.com
  - Causal Language Modeling: "inputs serve as labels too (just shifted by one
  element)"
  - "Shifting the inputs and labels to align them happens inside the model, so the data collator just copies the inputs to create the labels."

In [ ]:
from transformers import AutoTokenizer

# Load in Qwen2.5 tokenizer
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
# Tokenizer
  # link: https://huggingface.co/docs/transformers/en/main_classes/tokenizer

# Padding and truncation
  # link: https://huggingface.co/docs/transformers/pad_truncation
    # truncation, max_length, padding

# Process: Datasets, Batch
max_length = 512
def tokenize_fn(batch):
    return tokenizer(
        batch["final_prompt"],
        truncation=True,
        max_length=max_length,
        padding="max_length",
    )

tokenized = squad_dataset.map(
    tokenize_fn,
    batched=True,
    remove_columns=squad_dataset["train"].column_names
)


# Labels = input_ids (simple supervised fine-tune)
def add_labels(batch):
    # Just copy the list of lists; HF will convert to tensors later
    batch["labels"] = batch["input_ids"]
    return batch

tokenized = tokenized.map(add_labels, batched=True)

tokenized.set_format(type="torch")

train_ds = tokenized["train"]
eval_ds  = tokenized["validation"]
print(train_ds[0].keys()) # Now has input_ids, attention_mask & labels

Map:   0%|          | 0/60721 [00:00<?, ? examples/s]

Map:   0%|          | 0/7590 [00:00<?, ? examples/s]

Map:   0%|          | 0/7591 [00:00<?, ? examples/s]

Map:   0%|          | 0/60721 [00:00<?, ? examples/s]

Map:   0%|          | 0/7590 [00:00<?, ? examples/s]

Map:   0%|          | 0/7591 [00:00<?, ? examples/s]

dict_keys(['input_ids', 'attention_mask', 'labels'])


## Step 4: Load Qwen2.5 Model to GPU

In [ ]:
import os

print(os.listdir("/content/drive/MyDrive/SQuAD/models/qwen2_5_0_5B_squad_mc/checkpoint-7000"))

['config.json', 'generation_config.json', 'model.safetensors', 'chat_template.jinja', 'tokenizer_config.json', 'special_tokens_map.json', 'added_tokens.json', 'vocab.json', 'merges.txt', 'tokenizer.json', 'training_args.bin', 'optimizer.pt', 'scheduler.pt', 'rng_state.pth', 'trainer_state.json']


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# Load tokenizer
base_model_name = "Qwen/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)

model_path = "/content/drive/MyDrive/SQuAD/models/qwen2_5_0_5B_squad_mc/checkpoint-7000"
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True).to(device)

Using device: cuda


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

# Step 4: Load Qwen2.5 Model to GPU

In [ ]:
import torch
from transformers import AutoModelForCausalLM

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
)
model.to(device)

Using device: cpu


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2

# Step 5: Trainer

In [ ]:
from transformers import Trainer, TrainingArguments

output_dir = "/content/drive/MyDrive/SQuAD/models/qwen2_5_0_5B_squad_mc"

args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,
    warmup_ratio=0.1,
    eval_strategy="steps",
    eval_steps=500,
    save_steps=500,
    save_total_limit=2,
    logging_steps=50,
    logging_strategy="steps",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,
)

trainer.train()

/tmp/ipython-input-4229663554.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss,Validation Loss
500,0.930900,0.903881
1000,0.872800,0.855812
1500,0.787900,0.795777


Step,Training Loss,Validation Loss
500,0.930900,0.903881
1000,0.872800,0.855812
1500,0.787900,0.795777
2000,0.750500,0.732824
2500,0.684500,0.668327
3000,0.614400,0.605243
3500,0.545600,0.547842


# Step 6: Use Model

## Step 6.1: Load Model & Dataset

In [ ]:
!pip install -q transformers datasets accelerate

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# Load tokenizer
base_model_name = "Qwen/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(
    base_model_name,
    trust_remote_code=True
)

# Load in finetuned model & set to eval mode
model_path = "/content/drive/MyDrive/SQuAD/models/qwen2_5_0_5B_squad_mc/checkpoint-7000"
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True
)

model.to(device)
model.eval()

Using device: cuda


In [ ]:
from datasets import load_dataset

# Load in BBQ dataset from HuggingFace
bbq = load_dataset("walledai/BBQ")

## Step 6.2: Helper Functions

In [ ]:
letter_options = ["A", "B", "C"]  # BBQ has 3 choices

def build_prompt(example):
    '''Reformat BBQ examples into prompts for Qwen2.5'''
    context = example["context"]
    question = example["question"]
    choices = example["choices"] # list of 3 strings

    # Sanity check
    if len(choices) != 3:
        raise ValueError(f"Expected 3 choices, got {len(choices)}")

    options_block = ""
    for i in range(3):
        options_block += f"{letter_options[i]}. {choices[i]}\n"

    prompt = (
        f"Context:\n{context}\n\n"
        f"Question:\n{question}\n\n"
        f"Options:\n{options_block}\n\n"
        "Answer with the letter (A, B, or C) only.\n"
        "Answer:"
    )
    return prompt

In [ ]:
def predict_letter(example, max_new_tokens=2):
    '''Provide prompt to model and get the selected option'''

    letter_options = ["A", "B", "C"]  # BBQ has 3 choices

    # Build prompt & tokenize
    prompt = build_prompt(example)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Turn off gradient during prediction
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,  # greedy
            pad_token_id=tokenizer.eos_token_id,
        )

   # Extract only the newly generated tokens
    gen_ids = out[0][inputs["input_ids"].shape[1]:]
    gen_text = tokenizer.decode(gen_ids, skip_special_tokens=True).strip()

    if not gen_text:
        return None

    # Take first non-space character as answer
    letter = gen_text[0].upper()
    if letter not in letter_options:
        return None
    return letter

In [ ]:
def get_letter(example):
    """
    Convert BBQ's answer format into a letter (A, B, or C).
    Handles both integer labels (0,1,2) and string labels ('A','B','C').
    """

    letter_options = ["A", "B", "C"]  # BBQ has 3 choices

    # Original answer
    answer_val = example["answer"]

    # 0/1/2
    if isinstance(answer_val, int):
        if 0 <= answer_val < len(letter_options):
            return letter_options[answer_val]
        return None

    # A/B/C
    if isinstance(answer_val, str):
        ans = answer_val.strip().upper()
        if ans in letter_options:
            return ans
        return None

    return None

## Step 6.3: Save Results

In [ ]:
from tqdm.auto import tqdm

# BBQ's letter options
letter_options = ["A", "B", "C"]

category_results = {} # Per-category info
all_predictions = []  # Per-individual-example across each category

for category_name in bbq.keys():
    category_dataset = bbq[category_name]

    num_correct_predictions = 0
    num_evaluated_examples  = 0

    print(f"\nEvaluating category: {category_name} (n={len(category_dataset)})")

    for question_index_within_category, example in enumerate(tqdm(category_dataset)):

        # Get the correct answer letter (A/B/C)
        correct_answer_letter = get_letter(example)
        predicted_answer_letter = predict_letter(example)

        # Convert (A/B/C) to (0/1/2)
        correct_answer_index = letter_options.index(correct_answer_letter)
        predicted_answer_index = letter_options.index(predicted_answer_letter)

        # Check if correct or not
        prediction_is_correct = int(predicted_answer_index == correct_answer_index)
        num_evaluated_examples += 1
        if prediction_is_correct:
            num_correct_predictions += 1

        # Save example-specific info
        all_predictions.append({
            "category": category_name,
            "question_index": question_index_within_category,  # index within this category
            "context": example["context"],
            "question": example["question"],
            "choices": example["choices"],
            "true_answer_letter": correct_answer_letter,
            "true_answer_index": correct_answer_index,
            "predicted_answer_letter": predicted_answer_letter,
            "predicted_answer_index": predicted_answer_index,
            "correct": prediction_is_correct,
        })

    # Compute accuracy for this category
    accuracy = num_correct_predictions / num_evaluated_examples
    category_results[category_name] = {
        "accuracy": accuracy,
        "correct": num_correct_predictions,
        "total": num_evaluated_examples
    }

In [ ]:
import json
import os

save_path = "/content/drive/MyDrive/DATASCI266/SQuAD/bbq_runs_squad"
os.makedirs(save_path, exist_ok=True)

# 1) Save category-level summary
results_file = os.path.join(save_path, "qwen2_5_bbq_results.json")
with open(results_file, "w") as f:
    json.dump(category_results, f, indent=2)
print("Saved summary →", results_file)

# 2) Save per-example predictions (JSONL)
preds_file = os.path.join(save_path, "qwen2_5_bbq_predictions.jsonl")
with open(preds_file, "w") as f:
    for row in all_predictions:
        f.write(json.dumps(row) + "\n")
print("Saved per-example predictions →", preds_file)

Saved summary → /content/drive/MyDrive/DATASCI266/SQuAD/bbq_runs_squad/qwen2_5_bbq_results.json
Saved per-example predictions → /content/drive/MyDrive/DATASCI266/SQuAD/bbq_runs_squad/qwen2_5_bbq_predictions.jsonl
